# Land Cover Classification of Satellite Images

**Aidan O'Keefe**

IMAGE GOES HERE

## Overview

A deep learning (neural network) land cover classification project using satellite images (remote sensing).

## Business Understanding

## Data Understanding

In [95]:
#Import needed libraries
import os, shutil
from PIL import Image, ImageOps

#Standard Libraries
import numpy as np
import pandas as pd

# Visualizations
from matplotlib import pyplot as plt
import seaborn as sns

#TensorFlow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.data.experimental import cardinality
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense # creates densely connected layer object
from tensorflow.keras.layers import Flatten # takes 2D input and turns into 1D array
from tensorflow.keras.layers import Conv2D # convolution layer
from tensorflow.keras.layers import MaxPooling2D # max pooling layer
from tensorflow.keras.callbacks import EarlyStopping

### Import Data

In [84]:
#Set up directory path to image files
image_path = '/Users/Aidan/Documents/Flatiron/Phase_5/EuroSAT_RGB'

In [ ]:
#Split Images into Train, Test, and Validation folders
# OS and Shutil

In [85]:
# Normalize images
image_gen = ImageDataGenerator(rescale=1./255, validation_split = 0.2)

#Import data as 80% train and 20% test
train_generator = image_gen.flow_from_directory(image_path,
                                                class_mode = 'categorical', 
                                                subset ='training', 
                                                batch_size=128,
                                                  shuffle=True,
                                                seed=42)
                                               
test_generator= image_gen.flow_from_directory(image_path,
                                               class_mode= 'categorical',
                                               subset = "validation",
                                                batch_size=128,
                                                shuffle=True,
                                               seed=42)

Found 21600 images belonging to 10 classes.
Found 5400 images belonging to 10 classes.


In [86]:
#Confirm class balance for train and test

train_labels = train_generator.classes
test_labels = test_generator.classes

train_label, train_count = np.unique(train_labels, return_counts=True)
test_label, test_count = np.unique(test_labels, return_counts=True)

print('Train ~ {}'.format(list(zip(train_label, train_count))))
print('Test ~ {}'.format(list(zip(test_label, test_count))))

Train ~ [(0, 2400), (1, 2400), (2, 2400), (3, 2000), (4, 2000), (5, 1600), (6, 2000), (7, 2400), (8, 2000), (9, 2400)]
Test ~ [(0, 600), (1, 600), (2, 600), (3, 500), (4, 500), (5, 400), (6, 500), (7, 600), (8, 500), (9, 600)]


In [87]:
#Checking the classes in our train data
train_class_names = train_generator.class_indices
print('Train:', train_class_names)

#Checking the classes in our test data
test_class_names = test_generator.class_indices
print('Train:', test_class_names)

Train: {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}
Train: {'AnnualCrop': 0, 'Forest': 1, 'HerbaceousVegetation': 2, 'Highway': 3, 'Industrial': 4, 'Pasture': 5, 'PermanentCrop': 6, 'Residential': 7, 'River': 8, 'SeaLake': 9}


In [88]:
#Checking the number of classes in the train and test data match
len(test_class_names) == len(train_class_names)

True

## Modeling

### Train Test Split

In [89]:
# train_images, train_labels = next(train_generator)
# test_images, test_labels = next(test_generator)
# # val_images, val_labels = next(val_generator)

In [96]:
# m_train = train_images.shape[0]
# num_px = train_images.shape[1]
# m_test = test_images.shape[0]
# # m_val = val_images.shape[0]

# print ("Number of training samples: " + str(m_train))
# print ("Number of testing samples: " + str(m_test))
# # print ("Number of validation samples: " + str(m_val))
# print ("train_images shape: " + str(train_images.shape))
# print ("train_labels shape: " + str(train_labels.shape))
# print ("test_images shape: " + str(test_images.shape))
# print ("test_labels shape: " + str(test_labels.shape))
# # print ("val_images shape: " + str(val_images.shape))
# # print ("val_labels shape: " + str(val_labels.shape))

In [97]:
# train_img = train_images.reshape(train_images.shape[0], -1)
# test_img = test_images.reshape(test_images.shape[0], -1)
# # val_img = val_images.reshape(val_images.shape[0], -1)

# print(train_img.shape)
# print(test_img.shape)
# # print(val_img.shape)

In [93]:
# train_y = np.reshape(train_labels[:,0], (128,1))
# test_y = np.reshape(test_labels[:,0], (128,1))
# # val_y = np.reshape(val_labels[:,0], (200,1))

### Baseline Model

In [79]:
#Instantiate a Sequential model
baseline_model = Sequential()


# Input Layer- Convolution
baseline_model.add(Conv2D(filters=32,
                          kernel_size=(3, 3),
                          activation='relu',
                          input_shape= (256, 256, 3)))


# Layer 1- max pool in 2x2 window
baseline_model.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2- connect all nodes with dense layer
baseline_model.add(Flatten())
baseline_model.add(Dense(64, activation='relu'))

# Output Layer- softmax activiation for multi-categorical with 10 classes
baseline_model.add(Dense(10, activation='softmax'))

#Compile the sequential CNN model- adam optimizer, 
# categorical_crossentropy loss, and set our metric to accuracy
baseline_model.compile(optimizer='adam', 
                       loss='categorical_crossentropy',  
                       metrics=['accuracy'])

# print model summary
baseline_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 516128)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                33032256  
_________________________________________________________________
dense_13 (Dense)             (None, 10)                650       
Total params: 33,033,802
Trainable params: 33,033,802
Non-trainable params: 0
_________________________________________________________________


In [81]:
#Fit the model with the training data and 20% validation split (16% of original data).
baseline_history = baseline_model.fit(train_gen_dataset, 
                                      epochs = 5, 
                                      batch_size= 128, 
                                      verbose = 1) 
#                                       validation_split = 0.2)

Epoch 1/5
169/169 [==============================] - 340s 2s/step - loss: 2.6182 - accuracy: 0.3431
Epoch 2/5
169/169 [==============================] - 360s 2s/step - loss: 1.0948 - accuracy: 0.6223
Epoch 3/5
169/169 [==============================] - 323s 2s/step - loss: 0.8791 - accuracy: 0.6971
Epoch 4/5
169/169 [==============================] - 352s 2s/step - loss: 0.7732 - accuracy: 0.7324
Epoch 5/5
169/169 [==============================] - 319s 2s/step - loss: 0.6601 - accuracy: 0.7756


In [82]:
#Check loss and accuracy on test data
test_loss, test_acc = baseline_model.evaluate(test_gen_dataset, verbose = 1)

print('Test loss: ', test_loss)
print('Test accuracy: ', test_acc)

43/43 [==============================] - 25s 577ms/step - loss: 0.7964 - accuracy: 0.7154
Test loss:  0.7963578701019287
Test accuracy:  0.7153703570365906


It looks like after 5 Epochs our baseline model overfit with a 77.5% accuracy on the train data and 71.5% accuracy on the test data.

### First Model

In [ ]:
# # probability for each class
# y_proba = model_images.predict(x_test)
# y_proba

In [ ]:
# # argmax axis = -1 gets the column index of maximum probability for each row.
# # column index corresponds to digit classes (numbers 0 -9)
# predicted = np.argmax(y_proba, axis=-1)
# predicted

In [ ]:
# #View confusion matrix of test predictions
# cm_digits = confusion_matrix(y_test, predicted)
# disp = ConfusionMatrixDisplay(
#     confusion_matrix=cm_digits)

# disp.plot(cmap=plt.cm.Blues)
# plt.show()

CNN Sequential Model

In [ ]:
# #Instantiate the model
# model = Sequential()
# # define 3x3 filter window sizes. Create 32 filters.
# # COv2D input shape =(image_height, image_width, color_channels) for each image
# model.add(Conv2D(filters=32,
#                         kernel_size=(3, 3),
#                         activation='relu',
#                         input_shape=(28, 28, 1)))
# #IF STRIDE IS NOT SPECIFIED, IT IS AUTOMATICALLY THE KERNAL/FILTER SIZE. eg. s=3 for 3x3 filter
# # max pool in 2x2 window
# model.add(MaxPooling2D(pool_size=(2, 2)))
# #SAME DEFAULT FOR MAX POOL STRIDE
# # define 3x3 filter window sizes. Create 64 filters.
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))

# # transition to dense fully-connected part of network
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(10, activation='softmax'))

In [ ]:
# #Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])

In [ ]:
# # #Define Stopping Criteria 
# # valcallback = EarlyStopping(monitor='val_loss', mode='min', verbose = 1, patience = 2)


# # Fit the model
# history_cnn = \
# model.fit(train_images, train_labels, epochs= 20, validation_split = 0.2, batch_size=32, verbose = 2)

In [ ]:
# #Evaluate the accuracy of the model on test data
# _, test_acc = model.evaluate(test_images, to_categorical(test_labels), verbose =2)
# print(test_acc)

## Evaluation

## Conclusion

### Recommendations

### Next Steps